<a href="https://colab.research.google.com/github/digitalepidemiologylab/covid-twitter-bert/blob/master/CT_BERT_Huggingface_(GPU_training).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" width="450px" src="https://github.com/digitalepidemiologylab/covid-twitter-bert/raw/master/images/COVID-Twitter-BERT-medium.png">

# Finetuning COVID-Twitter-BERT using Huggingface
In this notebook we will finetune CT-BERT for sentiment classification using the transformer library by Huggingface.

Learn more about this library [here](https://huggingface.co/transformers/).

## Before proceeding
Create a copy of this notebook by going to "File - Save a Copy in Drive"


# Install transformers and import libraries

In [1]:
!pip install transformers

     |████████████████████████████████| 778kB 5.5MB/s 
     |████████████████████████████████| 890kB 15.3MB/s 
     |████████████████████████████████| 1.1MB 40.7MB/s 
     |████████████████████████████████| 3.0MB 54.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d717d2618d9a205fbddae8ca2ee7be9e7110728eadfbd8b187b3767fe071ad88
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features
)
import tensorflow as tf
import tensorflow_datasets as tfds
import json

# Choose a Model from the Huggingface Library

In [4]:
# Choose model
# @markdown >The default model is <i><b>COVID-Twitter-BERT</b></i>. You can however choose <i><b>BERT Base</i></b> or <i><b>BERT Large</i></b> to compare these models to the <i><b>COVID-Twitter-BERT</i></b>. All these three models will be initiated with a random classification layer. If you go directly to the Predict-cell after having compiled the model, you will see that it still runs the predition. However the output will be random. The training steps below will finetune this for the specific task. <br /><br /> 
model_name = 'digitalepidemiologylab/covid-twitter-bert' #@param ["digitalepidemiologylab/covid-twitter-bert", "bert-large-uncased", "bert-base-uncased"]

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Download the SST-2 Dataset and Prepare for Finetuning
You can skip this step if you are using the already finetuned model

In [5]:
# Paramteters
#@markdown >Batch size and sequence length needs to be set to prepare the data. The size of the batches depends on available memory. For Colab GPU limit batch size to 8 and sequence length to 96. By reducing the length of the input (max_seq_length) you can also increase the batch size. For a dataset like SST-2 with lots of short sentences. this will likely benefit training.
max_seq_length = 96 #@param {type: "integer"}
train_batch_size =  8#@param {type: "integer"} 
eval_batch_size = 8 #@param {type: "integer"}


#@markdown >The Glue dataset has around 62000 examples, and we really do not need them all for training a decent model. To cut down training time, please reduse this to only a percentage of the entire set.
use_percentage_of_data = 5 #@param {type: "slider", min: 1, max: 100}

# get dataset sizes
glue_builder = tfds.builder('glue/sst2')
num_train_examples = glue_builder.info.splits['train'].num_examples
num_dev_examples = glue_builder.info.splits['validation'].num_examples
num_labels = glue_builder.info.features['label'].num_classes

# download datasets and convert to training features
glue_builder.download_and_prepare()
train_data = glue_builder.as_dataset(split='train')
train_dataset = glue_convert_examples_to_features(train_data, tokenizer, max_length=max_seq_length, task='sst-2')
train_dataset = train_dataset.shuffle(100).batch(train_batch_size)

dev_data = glue_builder.as_dataset(split='validation')
dev_dataset = glue_convert_examples_to_features(dev_data, tokenizer, max_length=max_seq_length, task='sst-2')
dev_dataset = dev_dataset.shuffle(100).batch(eval_batch_size)

# Map the labels for printing
label_mapping = {i: glue_builder.info.features['label'].int2str(i) for i in range(num_labels)}

print(f'\n\nThe dataset is downloaded. The entire dataset has {num_train_examples + num_dev_examples} examples of which you are using {use_percentage_of_data}%. This will result in a train dataset with {int(num_train_examples * (use_percentage_of_data/100))} examples and a validation dataset with {int(num_dev_examples * (use_percentage_of_data/100))} examples.')

INFO:absl:Load pre-computed datasetinfo (eg: splits) from bucket.
INFO:absl:Loading info from GCS for glue/sst2/1.0.0
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/root/tensorflow_datasets/glue/sst2/1.0.0)


INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2FSST-2.zip?alt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8 into /root/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.cowOhVrpNUsvqdZqI70Nq3ISu63l9SOhTqYqoz6uEW3-Y.zipalt=media&token=aabc5f6b-e466-44a2-b9b4-cf6337f84ac8.tmp.5a488d8d16ea46da83cef961a62e14e4...
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
INFO:absl:Generating split train


Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incomplete9F37IY/glue-train.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/sst2/1.0.0.incomplete9F37IY/glue-train.tfrecord. Shard lengths: [67349]
INFO:absl:Generating split validation


Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incomplete9F37IY/glue-validation.tfrecord


...

Shuffling and writing examples to /root/tensorflow_datasets/glue/sst2/1.0.0.incomplete9F37IY/glue-test.tfrecord


INFO:absl:Done writing /root/tensorflow_datasets/glue/sst2/1.0.0.incomplete9F37IY/glue-test.tfrecord. Shard lengths: [1821]
INFO:absl:Skipping computing stats for mode ComputeStatsMode.AUTO.
INFO:absl:Constructing tf.data.Dataset for split train, from /root/tensorflow_datasets/glue/sst2/1.0.0


Dataset glue downloaded and prepared to /root/tensorflow_datasets/glue/sst2/1.0.0. Subsequent calls will reuse this data.


INFO:absl:Constructing tf.data.Dataset for split validation, from /root/tensorflow_datasets/glue/sst2/1.0.0




The dataset is downloaded. The entire dataset has 68221 examples of which you are using 5%. This will result in a train dataset with 3367 examples and a validation dataset with 43 examples.


# Compile the Model, Train it on the SST-2 Task and Save the Result
You can skip this step if you are using the already finetuned model

In [6]:
#@markdown >The default learning rate of 2e5 will be fine in most cases
learning_rate = 2e-5 #@param {type: "number"}

#@markdown > Typically these type of models are finetuned for 3 epochs. This can be increased for small datasets and decreased for large datasets.
num_epochs = 1  #@param {type: "integer"}

# Initialise a Model for Sequence Classification with 2 labels
config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, config=config)

# Optimizer and loss
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Metrics and callbacks
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
checkpoint_path = './checkpoints/checkpoint.{epoch:02d}'
callbacks = [tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True)]

# Compute some variables
train_steps_per_epoch = int(num_train_examples * (use_percentage_of_data/100) / train_batch_size)
dev_steps_per_epoch = int(num_dev_examples * (use_percentage_of_data/100) / eval_batch_size)


# Compile model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
history = model.fit(train_dataset, 
  epochs=num_epochs,
  steps_per_epoch=train_steps_per_epoch,
  validation_data=dev_dataset,
  validation_steps=dev_steps_per_epoch,
  callbacks=callbacks)

# Print some information about the training
print(f'\nThe training has finished training after {num_epochs} epochs.')
print('\nThe history contains the accuracy and loss at every epoch:')
print(json.dumps(history.history, indent=4))

print('\nThe checkpoint callback has generated a checkpoint after every epoch (loss being the training loss, val_loss is the validation loss):')
!ls -lha ./checkpoints/

print('\nWe will now save the finetuned model and the corresponding config file on your Colab disk.')
model.save_pretrained('./huggingface_model/')

print('\nTensorflow model and config-file is saved in ./huggingface_model/')
!ls -lha ./huggingface_model/

- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


420/420 [==============================] - 228s 542ms/step - loss: 0.3885 - accuracy: 0.8313 - val_loss: 0.2758 - val_accuracy: 0.9000

The training has finished training after 1 epochs.

The history contains the accuracy and loss at every epoch:
{
    "loss": [
        0.38849541544914246
    ],
    "accuracy": [
        0.831250011920929
    ],
    "val_loss": [
        0.2758089601993561
    ],
    "val_accuracy": [
        0.8999999761581421
    ]
}

The checkpoint callback have generated a checkpoint after every epoch:
total 3.8G
drwxr-xr-x 2 root root 4.0K Jul 17 19:01 .
drwxr-xr-x 1 root root 4.0K Jul 17 19:00 ..
-rw-r--r-- 1 root root   83 Jul 17 19:01 checkpoint
-rw-r--r-- 1 root root 6.3M Jul 17 19:00 checkpoint.01.data-00000-of-00002
-rw-r--r-- 1 root root 3.8G Jul 17 19:01 checkpoint.01.data-00001-of-00002
-rw-r--r-- 1 root root  87K Jul 17 19:01 checkpoint.01.index

We will now save the finetuned model and the corresponding config file on your Colab disk.

Tensorflow model

# Predict
Let's run some inference with the trained model

In [7]:
# Small function only used for formatting the output
def format_prediction(preds, label_mapping, label_name):
    preds = tf.nn.softmax(preds, axis=1)
    formatted_preds = []
    for pred in preds.numpy():
        # convert to Python types and sort
        pred = {label: float(probability) for label, probability in zip(label_mapping.values(), pred)}
        pred = {k: v for k, v in sorted(pred.items(), key=lambda item: item[1], reverse=True)}
        formatted_preds.append({label_name: list(pred.keys())[0], f'{label_name}_probabilities': pred})
    return formatted_preds

In [8]:
#@markdown >Please input text that the model can try to classify
input_text = 'Happy little clouds'  #@param {type: "string"}

# Tokenize the input 
input_ids = tf.constant(tokenizer.encode(input_text, add_special_tokens=True))[None, :]

# Run predictions
preds = model(input_ids)

# format logits
formatted_preds = format_prediction(preds[0], label_mapping, 'sentiment')

print(f'\nLabel Mapping:{json.dumps(label_mapping, indent=4)}')
print(f'\nLogits: {preds}')
print(f'\nProbabilities:{json.dumps(formatted_preds, indent=4)}')


Label Mapping:{
    "0": "negative",
    "1": "positive"
}

Logits: (<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.9338014,  0.8854352]], dtype=float32)>,)

Probabilities:[
    {
        "sentiment": "positive",
        "sentiment_probabilities": {
            "positive": 0.8604745268821716,
            "negative": 0.13952548801898956
        }
    }
]


##### Copyright 2020 Per Egil Kummervold and Martin Müller